# Estratégia de cobrança & LLM

#### Imports

In [23]:
import sys
import os

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)


In [24]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("seaborn-v0_8")
sns.set_palette("Set2")


#### Carregar clusters

In [25]:
from src.data_loader import load_raw_data
from src.features import process_pipeline
from src.clustering import run_clustering_pipeline

DATA_PATH = os.path.join(
    PROJECT_ROOT,
    "data",
    "raw",
    "credit_card_clients.xls"
)

df_raw = load_raw_data(DATA_PATH)
df_feat = process_pipeline(df_raw)

df_clustered = run_clustering_pipeline(df_feat, n_clusters=4)

df_clustered.head()


Tentando carregar: c:\Users\viras\OneDrive\Área de Trabalho\Workspace\GitHub\inadimplencia-segmentacao\data\raw\credit_card_clients.xls
✅ Arquivo detectado como Excel (.xls) e carregado com sucesso.
⚙️ Treinando KMeans com n_clusters=4...
📊 Silhouette Score: 0.3921


,limite_credito,genero,escolaridade,estado_civil,idade,status_pag_set,status_pag_ago,status_pag_jul,status_pag_jun,status_pag_mai,...,pay_amt6,is_inadimplente,score_gravidade,freq_atrasos,taxa_uso_limite,razao_pagamento_6m,canal_preferido,atende_cobranca,perfil_atendimento,cluster
0,20000,2,2,1,24,2,2,-1,-1,-2,...,0,1,8,2,0.195650,0.089434,WhatsApp,False,Evasivo,2
1,120000,2,2,2,26,-1,2,0,0,0,...,2000,1,8,2,0.022350,0.292791,Telefone,False,Evasivo,1
2,90000,2,2,2,34,0,0,0,0,0,...,5000,0,0,0,0.324878,0.108388,Telefone,False,Evasivo,2
3,50000,2,2,1,37,0,0,0,0,0,...,1000,0,0,0,0.939800,0.036259,Email,True,Engajado,2
4,50000,1,2,1,57,-1,0,-1,0,0,...,679,0,0,0,0.172340,0.540054,SMS,True,Engajado,0


#### Estratégia por cluster

In [26]:
estrategia_cluster = pd.DataFrame({
    "cluster": [0, 1, 2, 3],
    "perfil_risco": [
        "Baixo risco / Adimplente resiliente",
        "Risco leve / Pagador irregular",
        "Risco recorrente / Pressão financeira",
        "Alto risco / Crítico extremo"
    ],
    "objetivo_cobranca": [
        "Prevenção e educação financeira",
        "Regularização pontual",
        "Negociação ativa",
        "Mitigação de perda"
    ],
    "canal_prioritario": [
        "Email",
        "WhatsApp",
        "WhatsApp",
        "Telefone"
    ],
    "tom_comunicacao": [
        "Preventivo e cordial",
        "Objetivo e respeitoso",
        "Empático e negociador",
        "Firme e direto"
    ],
    "acao_recomendada": [
        "Lembrete + orientação",
        "Aviso de vencimento + CTA simples",
        "Proposta de parcelamento / acordo",
        "Escalonar para atendimento humano"
    ]
})

estrategia_cluster


,cluster,perfil_risco,objetivo_cobranca,canal_prioritario,tom_comunicacao,acao_recomendada
0,0,Baixo risco / Adimplente resiliente,Prevenção e educação financeira,Email,Preventivo e cordial,Lembrete + orientação
1,1,Risco leve / Pagador irregular,Regularização pontual,WhatsApp,Objetivo e respeitoso,Aviso de vencimento + CTA simples
2,2,Risco recorrente / Pressão financeira,Negociação ativa,WhatsApp,Empático e negociador,Proposta de parcelamento / acordo
3,3,Alto risco / Crítico extremo,Mitigação de perda,Telefone,Firme e direto,Escalonar para atendimento humano


#### System prompt

In [27]:
SYSTEM_PROMPT = """
Você é um assistente de cobrança bancária profissional.

REGRAS:
- Linguagem respeitosa e ética
- Nunca ameaçar ou constranger
- Nunca prometer descontos não autorizados
- Nunca mencionar termos técnicos ou internos
- Foco em clareza, empatia e solução
"""


#### Contexto estruturado do cliente

In [28]:
def build_customer_context(row):
    return f"""
PERFIL DO CLIENTE:
- Perfil de risco: {row['perfil_risco']}
- Canal prioritário: {row['canal_preferido']}
- Histórico financeiro:
  • Gravidade de atrasos: {row['score_gravidade']:.1f}
  • Frequência de atrasos: {row['freq_atrasos']:.1f}
  • Percentual pago (6 meses): {row['razao_pagamento_6m']:.0%}
  • Uso do limite: {row['taxa_uso_limite']:.0%}
"""


#### Instrução por cluster

In [29]:
PROMPTS_POR_CLUSTER = {
    0: "Gere uma mensagem curta, preventiva e cordial.",
    1: "Gere uma mensagem objetiva incentivando regularização.",
    2: "Gere uma mensagem empática propondo negociação.",
    3: "Gere uma mensagem firme solicitando contato com atendimento humano."
}


In [30]:
def gerar_prompt(row):
    contexto = build_customer_context(row)
    instrucao = PROMPTS_POR_CLUSTER[row["cluster"]]

    return f"""
{SYSTEM_PROMPT}

{contexto}

TAREFA:
{instrucao}

Gere apenas a mensagem final para o cliente.
"""


### Cliente Groq

In [31]:
import os
os.getenv("GROQ_API_KEY")


'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'

In [32]:
from groq import Groq
import os

client = Groq(
    api_key=os.getenv("GROQ_API_KEY")
)

#### Chamada do LLM

In [33]:
def gerar_mensagem_llm(prompt, model="llama-3.3-70b-versatile"):
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "Você é um assistente profissional de cobrança bancária."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.4,
        max_tokens=200
    )
    
    return response.choices[0].message.content.strip()


#### Tradução semântica dos clusters

In [35]:
# Mapeamento semântico dos clusters
cluster_labels = {
    0: "Baixo risco / Adimplente resiliente",
    1: "Risco leve / Pagador irregular",
    2: "Risco recorrente / Pressão financeira",
    3: "Alto risco / Crítico extremo"
}

df_clustered["perfil_risco"] = df_clustered["cluster"].map(cluster_labels)

df_clustered[["cluster", "perfil_risco"]].head()


,cluster,perfil_risco
0,2,Risco recorrente / Pressão financeira
1,1,Risco leve / Pagador irregular
2,2,Risco recorrente / Pressão financeira
3,2,Risco recorrente / Pressão financeira
4,0,Baixo risco / Adimplente resiliente


In [36]:
linha = df_clustered.sample(1, random_state=7).iloc[0]

prompt = gerar_prompt(linha)

print("PROMPT ENVIADO AO LLM:")
print("-" * 80)
print(prompt)

print("\nRESPOSTA DO LLM:")
print("-" * 80)
print(gerar_mensagem_llm(prompt))


PROMPT ENVIADO AO LLM:
--------------------------------------------------------------------------------


Você é um assistente de cobrança bancária profissional.

REGRAS:
- Linguagem respeitosa e ética
- Nunca ameaçar ou constranger
- Nunca prometer descontos não autorizados
- Nunca mencionar termos técnicos ou internos
- Foco em clareza, empatia e solução



PERFIL DO CLIENTE:
- Perfil de risco: Risco recorrente / Pressão financeira
- Canal prioritário: WhatsApp
- Histórico financeiro:
  • Gravidade de atrasos: 0.0
  • Frequência de atrasos: 0.0
  • Percentual pago (6 meses): 11%
  • Uso do limite: 79%


TAREFA:
Gere uma mensagem empática propondo negociação.

Gere apenas a mensagem final para o cliente.


RESPOSTA DO LLM:
--------------------------------------------------------------------------------
Olá! Entendo que você está passando por um momento financeiro desafiador. Quero ajudar a encontrar uma solução para regularizar sua situação. Você está disposto a conversar sobre possív